#### Imports

In [2]:




from itertools import product
import pickle


from Scripts.neuralnets.NNPreinstalledModelSelection import *


from Scripts.datahandling.BenchmarkDataset import BenchmarkDataset
from Scripts.datahandling.Dataloader import Dataloader
from Models.Autoencoder1 import Autoencoder



In [3]:

dataset = BenchmarkDataset(Project, signal_length, classes = list(range(1, 5 + 1)))
dataloaders, dataset_sizes = Dataloader(dataset, batch_size)
print(dataset_sizes)

{'train': 2448, 'val': 612}


***

#### Training Directly With a CNN

In [4]:
def model_initializer():
    model_type = 'resnet'; model_size = '18'; num_classes = 5; model_name = model_type + model_size
    model = model_selection(model_name, num_classes, pretrained = False)
    model.conv1 = nn.Conv2d(1, 64, kernel_size = (7, 7), stride = (2, 2), padding = (3, 3), bias = False)
    model = model.to(device)
    return model

def optimizer_initializer(model):
    return optim.Adam(model.parameters(), lr = 1e-3, weight_decay = 1e-5)

def scheduler_initializer(optimizer):
    return optim.lr_scheduler.StepLR(optimizer, step_size = 10, gamma = 0.1)

In [9]:
# optimizer = optim.SGD(model.parameters(), lr = 1e-3, momentum = 0.9)
criterion = nn.CrossEntropyLoss()

In [10]:
name = 'Resnet18 - Not Pretrained - 2 Classes - Adam'

model, stats = kfold_train(Project, model_initializer, dataset, criterion, optimizer_initializer, num_epochs = 50, num_folds = 5, batch_size = batch_size, log_name = name)

***

In [6]:
class HybridModel(nn.Module):
    def __init__(self, classifier, encoder):
        super(HybridModel, self).__init__()
        self.encoder = encoder
        self.classifier = classifier
        
    def forward(self, x):
        with torch.set_grad_enabled(False):
#             x = self.normalize(self.encoder(x))
            x = self.encoder(x)
            
        x = self.classifier(x)
        
        return x

In [21]:
model_type = 'resnet'; model_size = '18'; num_classes = 5; model_name = model_type + model_size
classifier = model_selection(model_name, num_classes)
classifier.conv1 = nn.Conv2d(1, 64, kernel_size = (7, 7), stride = (2, 2), padding = (3, 3), bias = False)
encoder = model.encoder
hybrid = HybridModel(classifier, encoder).to(device)

In [17]:
# optimizer = optim.SGD(hybrid.parameters(), lr = 1e-3, momentum = 0.9)
optimizer = optim. Adam(hy)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.5)
criterion = nn.CrossEntropyLoss()

In [ ]:
hybrid, stats = kfold_train(Project, hybrid, dataset, criterion, optimizer, num_epochs = 10, num_folds = 5)

***

In [11]:
class Autoencoder(nn.Module):
    def __init__(self, stride, kernel_n, n_feature_maps = 3):
        super(Autoencoder, self).__init__()
        n1 = kernel_n
        s = stride
        self.encoder = nn.Sequential(
            nn.Conv2d(1, n_feature_maps, kernel_size = (1, 2*n1 + 1), padding = (0, n1), stride = (1, s)),
            nn.ReLU(inplace = True),
        )
        
        self.pool    = nn.MaxPool2d(kernel_size = (1, 2), stride = (1, 2), return_indices = True)
        self.batch_norm = nn.BatchNorm2d(n_feature_maps)
        self.unpool  = nn.MaxUnpool2d(kernel_size = (1, 2), stride = (1, 2))
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(n_feature_maps, 1, kernel_size = (1, 2*n1 + 1), padding = (0, n1), stride = (1, s), output_padding = (0, s - 1))
        )
        
    
    def forward(self, x):
        x = self.encoder(x)
        x, _2 = self.pool(x)
        x = self.batch_norm(x)
        x = self.unpool(x, _2)
        x = self.decoder(x)
        
        return x
    
    def encode(self, x):
        with torch.set_grad_enabled(False):
            x = self.encoder(x)
            x, _ = self.pool(x)
            x = self.batch_norm(x)
            
        return x


In [12]:
def autoencoder_initializer(kernel_n, stride, n_feature_maps):
    model = Autoencoder(kernel_n = kernel_n, stride = stride, n_feature_maps = n_feature_maps).to(device)
    return model

In [13]:
def hybrid_initializer():
    classifier = model_selection(model_name, num_classes)
    classifier.conv1 = nn.Conv2d(fm, 64, kernel_size = (7, 7), stride = (2, 2), padding = (3, 3), bias = False)
    encoder = model.encoder
    hybrid = HybridModel(classifier, encoder).to(device)
    return hybrid

In [16]:
def optimizer_initializer(model):
    return optim.SGD(model.parameters(), lr = 1e-3, momentum = 0.9)

def scheduler_initializer(optimizer):
    return optim.lr_scheduler.StepLR(optimizer, step_size = 10, gamma = 0.5)

In [ ]:
S = [1]
K = [1, 3, 5]
FM = [1, 3]
model_type = 'resnet'; model_size = '18'; num_classes = 5; model_name = model_type + model_size

for (s, k, fm) in product(S, K, FM):
    # Autoencoder Training
    model = Autoencoder(kernel_n = k, stride = s, n_feature_maps = fm).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr = 1e-3, momentum = 0.9)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 10, gamma = 0.5)
    name = 'Autoencoder - s = {} k = {} fm = {} - 5 classes'.format(s, k, fm)
    model, stats = train_autoencoder(Project, model, dataloaders, dataset_sizes, criterion, optimizer, num_epochs = 60, log_name = name, scheduler = scheduler)
    
    with open('Outputs/'+name+' - stats', 'wb') as stats_file:
        pickle.dump(stats, stats_file)
        
    # Classifier training

    criterion = nn.CrossEntropyLoss()
    name = 'ResNet50 - N-TL - ' + name
    hybrid, stats = kfold_train(Project, hybrid_initializer, dataset, criterion, optimizer_initializer, num_epochs = 30, num_folds = 5, log_name = name)
    
    with open('Outputs/'+name+' - stats', 'wb') as stats_file:
        pickle.dump(stats, stats_file)
    
    del model
    del hybrid
    del stats